In [1]:
# Problem 1:
# Expected weekly demand for a product follows a triangular distribution with a min of 1000, max of 1800, and a mode of 1200.
# Your production capacity for a week follows a normal distribution with a mean of 1500 and a standard deviation of 200.
# You charge $40 per unit that you sell
# Your cost is $11 per unit that you produce
# Set a production target for the week that maximizes your expected profit.  Note that if your production target exceeds
# your capacity, then you will only pay for what you actually produce, but you will never produce less than your
# production target if you have capacity.
# STart by evaluating potential production levels of 1000, 1100, 1200, ..., 1700, 1800).  
# Once your model is running, adjust the 9 potential production levels to narrow in on 9 production levels within 
# increments of 10 (i.e. 1210, 1220, 1230, ... 1280,1290)
# Use 1000 iterations

# import needed packages
import numpy as np
import pandas as pd

# define constant values
price_per_unit = 40
cost_per_unit = 11
iterations = 1000
capacity_mean = 1500
capacity_std = 200
demand_min = 1000
demand_mode = 1200
demand_max = 1800

# create random values
def generate_random_values(iterations, capacity_mean, capacity_std, demand_min, demand_mode, demand_max):
    capacities = np.random.normal(capacity_mean, capacity_std, iterations)
    demands = np.random.triangular(demand_min, demand_mode, demand_max, iterations)
    return capacities, demands

#Compute the profits
def compute_profits(target, capacities, demands, price_per_unit, cost_per_unit):
    actual_productions = np.minimum(target, capacities)
    sales = np.minimum(actual_productions, demands)
    revenues = sales * price_per_unit
    costs = actual_productions * cost_per_unit
    profits = revenues - costs
    return np.mean(profits)


# Main simulation function
def simulate_production(target, iterations=1000):
    capacities, demands = generate_random_values(iterations, capacity_mean, capacity_std, demand_min, demand_mode, demand_max)
    return compute_profits(target, capacities, demands, price_per_unit, cost_per_unit)

# Initial evaluation of production levels
production_levels = range(1000, 1900, 100)
profits = [simulate_production(target) for target in production_levels]

# Create a DataFrame for better visualization
results_df = pd.DataFrame({'Production Target': production_levels, 'Expected Profit': profits})




#print the respective production targets and the associated expected profits

print("Initial Evaluation of Production Levels:")
print(results_df)






Initial Evaluation of Production Levels:
   Production Target  Expected Profit
0               1000     28983.038051
1               1100     31784.819279
2               1200     34101.390570
3               1300     35258.457770
4               1400     35555.764334
5               1500     35821.098213
6               1600     35549.306303
7               1700     35642.027952
8               1800     35235.862801


In [7]:
# Narrow down the production levels to increments of 10 within a promising range
narrowed_production_levels = range(1410, 1500, 10)
narrowed_profits = [simulate_production(target) for target in narrowed_production_levels]

# Create a DataFrame for better visualization
narrowed_results_df = pd.DataFrame({'Production Target': narrowed_production_levels, 'Expected Profit': narrowed_profits})

# Print the narrowed down production targets and the associated expected profits
print("\nNarrowed Down Evaluation of Production Levels:")
print(narrowed_results_df)

# Determine the optimal production target
optimal_target = narrowed_results_df.loc[narrowed_results_df['Expected Profit'].idxmax()]
print(f"\nOptimal Production Target: {optimal_target['Production Target']}")
print(f"Expected Profit at Optimal Target: ${optimal_target['Expected Profit']:.2f}")


Narrowed Down Evaluation of Production Levels:
   Production Target  Expected Profit
0               1410     35588.336358
1               1420     35818.919330
2               1430     35909.292806
3               1440     35666.657162
4               1450     35915.368034
5               1460     35793.205152
6               1470     36131.229508
7               1480     35722.718355
8               1490     35576.801859

Optimal Production Target: 1470.0
Expected Profit at Optimal Target: $36131.23


In [ ]:
check point

In [1]:
# Problem 2:  
# Recall the tables and chairs problem from week 10 lecture (maybe even copy and paste as a starting point)
# You have expanded your operations to include benches.  You have also adjusted some of your designs.
# Chairs require 4 short boards and 2 medium boards (profit margin of $30)
# Benches require 4 long boards, 3 medium boards and 5 short boards (profit margin of $40)
# Tables require 6 long boards, 2 medium boards and 4 short boards (profit margin of $80)
# You must produce at least 3 of each type of furniture
# You have 120 short boards, 85 medium boards and 105 long boards to work with.  
# you cannot cut a longer board to make a shorter board (although I breifly considered allowing it to make for a 
# more complicated problem)
# How many of each product should you make, and what is your profit?




from pulp import *

# Define the problem
prob = LpProblem("Furniture_Production", LpMaximize)

# Define the variables
Chairs = LpVariable("Chairs", lowBound=3, cat='Integer')  # At least 3 chairs
Benches = LpVariable("Benches", lowBound=3, cat='Integer')  # At least 3 benches
Tables = LpVariable("Tables", lowBound=3, cat='Integer')  # At least 3 tables

# Define the objective function
prob += 30*Chairs + 40*Benches + 80*Tables, "Total_Profit"

# Define the constraints
prob += 4*Chairs + 5*Benches + 4*Tables <= 120, "Short_Boards"
prob += 2*Chairs + 3*Benches + 2*Tables <= 85, "Medium_Boards"
prob += 0*Chairs + 4*Benches + 6*Tables <= 105, "Long_Boards"

# Solve the problem
status = prob.solve()

# Print the results
print("Status:", LpStatus[status])
print("Number of Chairs:", value(Chairs))
print("Number of Benches:", value(Benches))
print("Number of Tables:", value(Tables))
print("Total Profit:", value(prob.objective))


Status: Optimal
Number of Chairs: 11.0
Number of Benches: 3.0
Number of Tables: 15.0
Total Profit: 1650.0


In [1]:
# Problem 3:
# resolve the warehouse location example from class session 11, with two warehouses.  
# You can copy and paste the class example in here and just modify as needed.

import numpy as np
from scipy.optimize import minimize

def calcDistance(x):
    lat1, long1, lat2, long2 = x
    citylat = np.array([40.7, 42.3, 40, 35.2, 33.8, 30, 25.8, 32.8, 29.8, 41.8, 42.4, 41.5, 39.8, 39.8, 45, 33.5, 40.8, 34.1, 37.8, 32.8, 47.6, 51])
    citylong = np.array([-73.9, -71, -75.1, -80.8, -84.4, -89.9, -80.2, -96.8, -95.4, -87.7, -83.1, -81.7, -86.1, -104.9, -93.3, -112.1, -111.9, -118.4, -122.6, -117.1, -122.4, -114.1])
    shipments = np.array([15, 8, 10, 6, 11, 8, 13, 10, 12, 14, 11, 8, 7, 8, 9, 11, 10, 18, 12, 10, 13, 5])
    
    # Calculate distances to both warehouses
    dist1 = np.sqrt((citylat - lat1) ** 2 + (citylong - long1) ** 2)
    dist2 = np.sqrt((citylat - lat2) ** 2 + (citylong - long2) ** 2)
    
    # Use the minimum distance to each city
    distshipped = shipments * np.minimum(dist1, dist2)
    Distance = np.sum(distshipped)
    
    return Distance

def objective(x):
    return calcDistance(x)

def constraint(x):
    return 0 + calcDistance(x)

cons = ({'type': 'ineq', 'fun': constraint})

latguess1 = 40
longguess1 = -70
latguess2 = 35
longguess2 = -95

x0 = np.array([latguess1, longguess1, latguess2, longguess2])

sol = minimize(objective, x0, method='SLSQP', constraints=cons, options={'disp': True})

LatOpt1 = sol.x[0]
LongOpt1 = sol.x[1]
LatOpt2 = sol.x[2]
LongOpt2 = sol.x[3]
DistOpt = sol.fun

print('The minimal distance is ' + str(round(DistOpt, 2)))
print('Warehouse 1 Latitude:', round(LatOpt1, 2))
print('Warehouse 1 Longitude:', round(LongOpt1, 2))
print('Warehouse 2 Latitude:', round(LatOpt2, 2))
print('Warehouse 2 Longitude:', round(LongOpt2, 2))


Optimization terminated successfully    (Exit mode 0)
            Current function value: 1882.5367742178264
            Iterations: 11
            Function evaluations: 61
            Gradient evaluations: 11
The minimal distance is 1882.54
Warehouse 1 Latitude: 38.16
Warehouse 1 Longitude: -84.03
Warehouse 2 Latitude: 35.93
Warehouse 2 Longitude: -117.22
